In [2]:
import pandas as pd

In [3]:
data = pd.read_csv("../data/jaguar.csv",index_col=0)
data.head()

,body,topic
0,Ягуар (Panthera onca) — вид хищных млекопитающ...,животное
1,На вид этот американский зверь - вылитый леопа...,животное
2,"Ягуар – хищник семейства кошачьих, принадлежащ...",животное
3,Название: название ягуара предположительно про...,животное
4,Ягуар относится к одним из самых крупных живот...,животное


In [4]:
data.topic.unique()

array(['животное', 'напиток', 'машина', 'фильм', 'швейная машина'], dtype=object)

In [6]:
import re
from nltk.corpus import stopwords
from pymystem3 import Mystem

regex = re.compile("[А-Яа-я-]+")
mystopwords = stopwords.words('russian')
m = Mystem()

def words_only(text, regex=regex):
    return " ".join(regex.findall(text))

def  remove_stopwords(text, mystopwords = mystopwords):
    try:
        return " ".join([token for token in text.split() if not token in mystopwords])
    except:
        return ""
    
def lemmatize(text, mystem=m):
    try:
        return "".join(m.lemmatize(text)).strip()  
    except:
        return " "




data.body = data.body.str.lower()
data.body = data.body.apply(words_only)
data.body = data.body.apply(remove_stopwords)   
data.body = data.body.apply(lemmatize)

#### Задание 1
1. Найдите топ-10 слов по частоте в разделе 'машина'
2. Найдите топ-10 слов по $tf-idf$ в разделе 'машина'

In [8]:
texts = []
for i in range(len(data)):
    tokens = data.body.iloc[i].split()
    texts.append(tokens)

In [9]:
from gensim import corpora, models, similarities
import numpy as np

dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts] 
tfidf = models.TfidfModel(corpus, id2word=dictionary)
corpus_tfidf = tfidf[corpus]
index = similarities.MatrixSimilarity(corpus_tfidf)
sims = index[corpus_tfidf]

Using TensorFlow backend.


In [18]:
doc = "машина" ##  документ – запрос
vec_bow = dictionary.doc2bow(doc.lower().split())
sims = index[vec_bow] ##  индексируем вектор документ и находим ближайшие документы 
sims = sorted(enumerate(sims), key=lambda item: -item[1])

#### Задание 2
1. В скольких документах встречается слово 'машина'?
2. В скольких документах встречаются слова 'быстрый' и 'машина'?
3. В скольких документах встречается словосочетание 'быстрая машина' (или 'быстрый машина', если мы ищем по леммам?)